In [19]:
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score

In [20]:
#loading the diabetes dataset in a pandas dataset
dataset=pd.read_csv('final yr clinical.csv')

In [21]:
#printing the first rows of our dataset
dataset.head()

,subject_id,Age_cat,BMI_cat,HEART,LUNG,KIDNEY,DIABETES,HTN,IMMUNO,Resp_Symp,...,ddimer_3_cat,ldh_3_cat,abs_neut_7_cat,abs_lymph_7_cat,abs_mono_7_cat,creat_7_cat,crp_7_cat,ddimer_7_cat,ldh_7_cat,Class Label
0,1,1,4,0,0,0,0,0,0,1,...,1,1,-1,-1,-1,-1,-1,-1,-1,0
1,2,2,2,0,0,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
2,3,3,4,0,1,0,0,0,0,1,...,2,3,-1,-1,-1,-1,-1,-1,-1,0
3,4,1,2,0,0,0,0,0,0,1,...,2,3,-1,-1,-1,-1,-1,-1,-1,0
4,5,3,3,0,0,0,1,1,0,1,...,3,-1,3,5,3,1,4,4,3,0


In [22]:
dataset.describe()

,subject_id,Age_cat,BMI_cat,HEART,LUNG,KIDNEY,DIABETES,HTN,IMMUNO,Resp_Symp,...,ddimer_3_cat,ldh_3_cat,abs_neut_7_cat,abs_lymph_7_cat,abs_mono_7_cat,creat_7_cat,crp_7_cat,ddimer_7_cat,ldh_7_cat,Class Label
count,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,...,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000
mean,193.330729,3.218750,2.479167,0.184896,0.276042,0.158854,0.361979,0.518229,0.109375,0.953125,...,1.119792,0.927083,0.882812,0.875000,0.893229,0.539062,0.580729,0.533854,0.148438,0.127604
std,111.795820,1.206623,1.151493,0.388720,0.447621,0.366017,0.481200,0.500319,0.312516,0.211647,...,2.082182,2.101820,2.101676,2.185879,2.214728,1.769641,2.206978,2.207015,1.845564,0.334084
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,96.750000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
50%,193.500000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,2.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
75%,290.250000,4.000000,3.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,3.000000,2.000000,0.000000
max,386.000000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000


In [29]:
X=dataset.drop(columns='Class Label',axis=1)
Y=dataset['Class Label']

In [30]:
X

,subject_id,Age_cat,BMI_cat,HEART,LUNG,KIDNEY,DIABETES,HTN,IMMUNO,Resp_Symp,...,crp_3_cat,ddimer_3_cat,ldh_3_cat,abs_neut_7_cat,abs_lymph_7_cat,abs_mono_7_cat,creat_7_cat,crp_7_cat,ddimer_7_cat,ldh_7_cat
0,1,1,4,0,0,0,0,0,0,1,...,1,1,1,-1,-1,-1,-1,-1,-1,-1
1,2,2,2,0,0,0,0,0,0,1,...,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,3,4,0,1,0,0,0,0,1,...,3,2,3,-1,-1,-1,-1,-1,-1,-1
3,4,1,2,0,0,0,0,0,0,1,...,2,2,3,-1,-1,-1,-1,-1,-1,-1
4,5,3,3,0,0,0,1,1,0,1,...,5,3,-1,3,5,3,1,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,346,3,2,0,0,0,0,0,0,1,...,5,3,-1,3,3,4,3,5,4,2
380,348,4,2,0,0,1,0,1,0,1,...,5,4,-1,3,2,1,3,4,5,-1
381,354,4,3,1,1,0,0,1,0,1,...,4,3,3,3,3,2,1,4,2,2
382,365,5,2,0,1,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [31]:
y

0      0
1      0
2      0
3      0
4      0
      ..
379    1
380    1
381    1
382    1
383    1
Name: Class Label, Length: 384, dtype: int64

In [32]:
X.drop(columns='subject_id',axis=1)

,Age_cat,BMI_cat,HEART,LUNG,KIDNEY,DIABETES,HTN,IMMUNO,Resp_Symp,Fever_Sympt,...,crp_3_cat,ddimer_3_cat,ldh_3_cat,abs_neut_7_cat,abs_lymph_7_cat,abs_mono_7_cat,creat_7_cat,crp_7_cat,ddimer_7_cat,ldh_7_cat
0,1,4,0,0,0,0,0,0,1,1,...,1,1,1,-1,-1,-1,-1,-1,-1,-1
1,2,2,0,0,0,0,0,0,1,1,...,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,4,0,1,0,0,0,0,1,0,...,3,2,3,-1,-1,-1,-1,-1,-1,-1
3,1,2,0,0,0,0,0,0,1,1,...,2,2,3,-1,-1,-1,-1,-1,-1,-1
4,3,3,0,0,0,1,1,0,1,1,...,5,3,-1,3,5,3,1,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,3,2,0,0,0,0,0,0,1,1,...,5,3,-1,3,3,4,3,5,4,2
380,4,2,0,0,1,0,1,0,1,1,...,5,4,-1,3,2,1,3,4,5,-1
381,4,3,1,1,0,0,1,0,1,0,...,4,3,3,3,3,2,1,4,2,2
382,5,2,0,1,0,0,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [33]:
X=X.to_numpy()

In [34]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [35]:
print(X.shape,X_train.shape,X_test.shape)

(384, 38) (307, 38) (77, 38)


In [96]:
classifier = xgb.XGBClassifier(random_state = 12345,learning_rate = 0.1, min_child_weight=10, max_depth =5, gamma=1,min_samples_split = 0.1, n_estimators = 100, subsample = 1.0)

In [97]:
classifier.fit(X_train,Y_train)

[22:10:04] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:10:04] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\baibh\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=1, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_weight=10, min_samples_split=0.1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=12345, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1.0, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [98]:
X_train_prediction=classifier.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [99]:
print("Accuracy is ",training_data_accuracy)

Accuracy is  0.9381107491856677


In [100]:
X_test_prediction=classifier.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [101]:
print("Accuracy is",test_data_accuracy)

Accuracy is 0.8701298701298701
